# Sentiment analysis

In [10]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Define the training data
train_data = [
    ("Hi, I would like to know about the security settings", "security"),
    ("I want to order some italian food", "ordering"),
    ("What time does the store open tomorrow?", "information"),
    ("Can you help me find a hotel near the airport?", "help"),
]

# Define sub-level labels for each top-level label
sub_labels = {
    "security": ["security_relating_to", "security_concerns"],
    "ordering": ["ordering_type_of_food", "ordering_delivery"],
    "information": ["information_about", "information_schedule"],
    "help": ["help_with_finding", "help_with_booking"]
}

# Define the mapping between top-level labels and integers
top_level_label_map = {label: i for i, label in enumerate(set([data[1] for data in train_data]))}

# Define the mapping between sub-level labels and integers
sub_level_label_map = {label: i for i, label in enumerate(set([sub_label for sub_labels_list in sub_labels.values() for sub_label in sub_labels_list]))}

# Convert the training data labels to integers using the label_map and sub_label_map
# A tensor is a multi-dimensional array that looks like a numpy array, it's used for neural networks
top_level_labels = torch.tensor([top_level_label_map[data[1]] for data in train_data])
sub_level_labels = torch.tensor([sub_level_label_map[sub_label] for label, sub_labels_list in sub_labels.items() for sub_label in sub_labels_list for data in train_data if data[1] == label])

# Load the pre-trained BERT model and tokenizer
top_level_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(top_level_label_map))
sub_level_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(sub_level_label_map))
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the training data and convert to tensors
top_level_inputs = tokenizer.batch_encode_plus([data[0] for data in train_data], padding=True, truncation=True, return_tensors="pt")
sub_level_inputs = tokenizer.batch_encode_plus([data[0] for data in train_data for sub_label in sub_labels[data[1]]], padding=True, truncation=True, return_tensors="pt")

# Fine-tune the models on the training data
top_level_optimizer = torch.optim.Adam(top_level_model.parameters(), lr=1e-5)
sub_level_optimizer = torch.optim.Adam(sub_level_model.parameters(), lr=1e-5)
for epoch in range(10):
    top_level_outputs = top_level_model(top_level_inputs["input_ids"], attention_mask=top_level_inputs["attention_mask"], labels=top_level_labels)
    top_level_loss = top_level_outputs.loss
    top_level_loss.backward()
    top_level_optimizer.step()
    top_level_optimizer.zero_grad()

    sub_level_outputs = sub_level_model(sub_level_inputs["input_ids"], attention_mask=sub_level_inputs["attention_mask"], labels=sub_level_labels)
    sub_level_loss = sub_level_outputs.loss
    sub_level_loss.backward()
    sub_level_optimizer.step()
    sub_level_optimizer.zero_grad()

    # print(f"Epoch {epoch+1}, Top-Level Loss: {top_level_loss.item()}, Sub-Level Loss: {sub_level_loss.item()}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Sentiment analysis query function

In [11]:
# Test the model
text = "What time does the Italian restaurant open tomorrow?"

def predict_intent(text):
    top_level_inputs = tokenizer.encode_plus(text, padding=True, truncation=True, return_tensors="pt")
    top_level_outputs = top_level_model(top_level_inputs["input_ids"], attention_mask=top_level_inputs["attention_mask"])
    top_level_predicted_label = torch.argmax(top_level_outputs.logits).item()
    top_level_predicted_intent = [k for k, v in top_level_label_map.items() if v == top_level_predicted_label][0]
    
    sub_level_inputs = tokenizer.encode_plus(text, padding=True, truncation=True, return_tensors="pt")
    sub_level_outputs = sub_level_model(sub_level_inputs["input_ids"], attention_mask=sub_level_inputs["attention_mask"])
    sub_level_predicted_label = torch.argmax(sub_level_outputs.logits).item()
    sub_level_predicted_intent = [k for k, v in sub_level_label_map.items() if v == sub_level_predicted_label][0]
    
    return top_level_predicted_intent, sub_level_predicted_intent

top_level_intent, sub_level_intent = predict_intent(text)
print(top_level_intent, sub_level_intent)

information information_about


## GPT

In [ ]:
# GPT model herel

In [ ]:
# GPT query function here

## Gradio

In [22]:
import gradio as gr
import time
import random

# load the pre-trained intent analysis model
# nlp = spacy.load("en_trf_bertbaseuncased_lg")

response_map = {
    ("security", "security_relating_to"): ["Our security measures include...", "We take security very seriously and have implemented..."],
    ("security", "security_concerns"): ["We understand your security concerns and have taken steps to address them.", "You can trust that your information is safe with us."],
    ("ordering", "ordering_type_of_food"): ["Our menu features a variety of Italian dishes, including pizza and pasta.", "We also offer a selection of salads and appetizers."],
    ("ordering", "ordering_delivery"): ["You can place a delivery order on our website or by calling our delivery hotline.", "Delivery is available within a 10-mile radius of our store."],
    ("information", "information_about"): ["Our store offers a variety of products, including...", "We also have a rewards program that allows you to earn points on your purchases."],
    ("information", "information_schedule"): ["We are open from 9am to 10pm, 7 days a week.", "Our business hours are 9am to 5pm, Monday to Friday."],
    ("help", "help_with_finding"): ["Here are some hotels near the airport:...", "I can help you find a hotel that meets your needs."],
    ("help", "help_with_booking"): ["You can book a room on our website or by calling our reservation hotline.", "We also offer a loyalty program that gives you discounts on future bookings."],
    ("information", "ordering"): ["You can place an order on our website or by calling our order hotline.", "We also offer a loyalty program that gives you discounts on future orders."]
}

# Example usage
intent = "information"
sub_intent = "ordering_delivery"

try:
    response = random.choice(response_map[(intent, sub_intent)])
except KeyError:
    response = "I'm sorry, I don't have a response for that."

print(response)


with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]
        predicted_intent = predict_intent(user_message)
        # Random choice randomly chooses one of the options that matches the intent
     
        response = random.choice(response_map[predicted_intent])
        history[-1][1] = response
        # The sleep is to simulate a more natural conversation
        time.sleep(1)
        return history



    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch()


I'm sorry, I don't have a response for that.
Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Program Files\Python38\lib\site-packages\gradio\routes.py", line 395, in run_predict
    output = await app.get_blocks().process_api(
  File "c:\Program Files\Python38\lib\site-packages\gradio\blocks.py", line 1193, in process_api
    result = await self.call_function(
  File "c:\Program Files\Python38\lib\site-packages\gradio\blocks.py", line 916, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Program Files\Python38\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Program Files\Python38\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "c:\Program Files\Python38\lib\site-packages\anyio\_backends\_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "C:\Users\Frank\AppData\Local\Temp\ipykernel_20876\3935284218.py", line 45, in bot
    response

a

#### Source:

 https://www.section.io/engineering-education/intent-classification-with-rasa-and-spacy/